# 📊 Preparación del Conjunto de Datos Histórico de BTC/USD

En este notebook **crearemos el conjunto de datos histórico de la criptomoneda Bitcoin (BTC) frente al dólar estadounidense (USD)**.
Este dataset estará listo para su uso en el entrenamiento y la modelización de redes neuronales, con el objetivo de predecir el movimiento diario del precio de BTC/USD.

In [1]:
# =============================================================================
# LIBRERIAS
# =============================================================================

import pandas as pd
import numpy as np
import ta

## 🔹 Importación de Datos

En esta sección importaremos los datos históricos de Bitcoin frente al dólar estadounidense (BTC/USD).  
Los datos han sido obtenidos de [Kaggle: Comprehensive BTC/USD 1m Data](https://www.kaggle.com/datasets/imranbukhari/comprehensive-btcusd-1m-data/data).

Para nuestro análisis utilizaremos el archivo **`BTCUSD_1m_Binance.csv`**, que, aunque cubre un período más corto (desde 2017), incluye **variables adicionales** como `Number of trades`, `Taker buy base asset volume` y `Taker buy quote asset volume`, que podrían resultar útiles para la predicción del movimiento diario del precio de BTC/USD. Además, al centrarnos en un período más reciente, este dataset **refleja mejor el comportamiento actual del mercado**, evitando introducir ruido o patrones históricos menos relevantes para nuestras predicciones en la actualidad.

In [2]:
# =============================================================================
# LECTURA CSV
# =============================================================================

df_binance_1m = pd.read_csv("BTCUSD_1m_Binance.csv")
df_binance_1m.head()

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,2017-08-17 04:00:00,4261.48,4261.48,4261.48,4261.48,1.775183,2017-08-17 04:00:59.999,7564.906851,3.0,0.075183,320.390851,0.0
1,2017-08-17 04:01:00,4261.48,4261.48,4261.48,4261.48,0.000000,2017-08-17 04:01:59.999,0.000000,0.0,0.000000,0.000000,0.0
2,2017-08-17 04:02:00,4280.56,4280.56,4280.56,4280.56,0.261074,2017-08-17 04:02:59.999,1117.542921,2.0,0.261074,1117.542921,0.0
3,2017-08-17 04:03:00,4261.48,4261.48,4261.48,4261.48,0.012008,2017-08-17 04:03:59.999,51.171852,3.0,0.012008,51.171852,0.0
4,2017-08-17 04:04:00,4261.48,4261.48,4261.48,4261.48,0.140796,2017-08-17 04:04:59.999,599.999338,1.0,0.140796,599.999338,0.0


### 📌 Descripción de las Columnas del Dataset

A continuación se detallan las columnas presentes en el archivo **`BTCUSD_1m_Binance.csv`**:

- **`Open time`**: Timestamp de apertura de la vela. Indica el momento exacto en que comenzó el intervalo de tiempo (por ejemplo, 1 minuto) de la vela.

- **`Open`**: Precio de apertura de la vela. Es el primer precio registrado en ese intervalo.

- **`High`**: Precio máximo alcanzado durante la vela.

- **`Low`**: Precio mínimo alcanzado durante la vela.

- **`Close`**: Precio de cierre de la vela. Es el último precio registrado en ese intervalo.

- **`Volume`**: Cantidad de BTC negociada durante la vela.

- **`Close time`**: Timestamp de cierre de la vela. Representa el final del intervalo de la vela.

- **`Quote asset volume`**: Volumen negociado en la moneda cotizada (por ejemplo, USD) durante la vela.

- **`Number of trades`**: Número total de operaciones realizadas durante la vela.

- **`Taker buy base asset volume`**: Volumen de BTC comprado por los **takers** (operadores que aceptan el precio de mercado) durante la vela.

- **`Taker buy quote asset volume`**: Volumen en la moneda cotizada comprado por los **takers**.

- **`Ignore`**: Columna reservada por Binance, generalmente sin uso (valores nulos o cero).

> 💡 Para la predicción diaria, las columnas más relevantes suelen ser: `Open`, `High`, `Low`, `Close`, `Volume`, y opcionalmente las relacionadas con actividad de mercado (`Quote asset volume`, `Number of trades`, `Taker buy ...`).


## 🔹 Procesado

En esta sección realizaremos el **procesado de los datos** para preparar el dataset para su análisis y modelización.  
Durante este paso, **generaremos nuevas variables derivadas** que aporten información adicional sobre el comportamiento del mercado y de la blockchain, lo que nos permitirá identificar las más relevantes para la predicción y mejorar la precisión de nuestro modelo.

### 📌 Columnas descartadas

Durante el procesado, **descartaremos las columnas `Ignore` y `Close time`**:  

- **`Ignore`**: columna reservada por Binance que no contiene información útil (valores nulos o siempre cero).  
- **`Close time`**: timestamp redundante, ya que se puede derivar directamente de `Open time` y la duración de la vela; no aporta valor adicional para nuestra predicción diaria.

### 📌 Agregación Diaria

Nuestro objetivo es **predecir si el precio de BTC/USD subirá o bajará al día siguiente**, por lo que necesitamos trabajar con datos agregados a nivel diario.  
Para ello, transformaremos los datos originales de 1 minuto a **velas diarias**, calculando:  

- `Open`: primer precio del día  
- `High`: precio máximo del día  
- `Low`: precio mínimo del día  
- `Close`: precio de cierre del día  
- `Volume`: volumen total negociado en BTC  
- `Quote asset volume`: volumen total en moneda cotizada  
- Otras variables relevantes (`Number of trades`, `Taker buy ...`) agregadas por día  

Esta agregación diaria nos permite **resumir la información relevante del mercado** y preparar el dataset de manera coherente con nuestro objetivo de predicción diaria.

In [3]:
# =============================================================================
# AGREGACION DIARIA
# =============================================================================

# Convertimos la columna 'Open time' a tipo datetime para poder manipular fechas
df_binance_1m['Open time'] = pd.to_datetime(df_binance_1m['Open time'])

# Ponemos 'Open time' como índice temporal para usar resample
df_binance_1m.set_index('Open time', inplace=True)

# Reagrupar a diario usando 'resample' y aplicar distintas funciones según la columna
df_binance_1d = df_binance_1m.resample('1D').agg({
    'Open': 'first',                      # Primer precio del día
    'High': 'max',                        # Precio máximo del día
    'Low': 'min',                         # Precio mínimo del día
    'Close': 'last',                      # Último precio del día
    'Volume': 'sum',                       # Suma del volumen de todas las velas del día
    'Quote asset volume': 'sum',           # Suma del volumen en la moneda cotizada
    'Number of trades': 'sum',             # Suma del número de trades diarios
    'Taker buy base asset volume': 'sum',  # Suma del volumen comprado por takers
    'Taker buy quote asset volume': 'sum'  # Suma de la moneda cotizada comprada por takers
})

# Resetear el índice para que 'Open time' vuelva a ser columna
df_binance_1d = df_binance_1d.reset_index()

# Guardamos el dataframe diario resultante en un nuevo CSV
#df_binance_1d.to_csv("datos_binance_diario.csv", index=False)

df_binance_1d.head()

,Open time,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
0,2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,3.454770e+06,3427.0,616.248541,2.678216e+06
1,2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,5.086958e+06,5233.0,972.868710,4.129123e+06
2,2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,1.549484e+06,2153.0,274.336042,1.118002e+06
3,2017-08-20,4139.98,4211.08,4032.62,4086.29,467.083022,1.930364e+06,2321.0,376.795947,1.557401e+06
4,2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,2.797232e+06,3972.0,557.356107,2.255663e+06


## 📌 Creación de la Variable Objetivo: Dirección del Precio

Siguiendo la metodología propuesta por Livieris en su artículo *"An Advanced CNN-LSTM Model for Cryptocurrency Forecasting"* [(MDPI, 2021)](https://www.mdpi.com/2079-9292/10/3/287), consideramos **movimientos direccionales del precio** en lugar de la magnitud del mismo. De este modo, nuestro problema se convierte en un **problema de clasificación binaria**:

- **Clase 1**: El precio de Bitcoin aumenta del día `t` al día `t+1` (`Close_t+1 > Close_t`) y se codifica como `1`.
- **Clase 0**: El precio de Bitcoin disminuye o se mantiene igual (`Close_t+1 ≤ Close_t`) y se codifica como `0`.


Este enfoque también ha sido aplicado y comparado empíricamente en estudios como *"Deep learning for Bitcoin price direction prediction: models and trading strategies empirically compared"* de Oluwadamilare Omole y David Enke [(Springer, 2024)](https://jfin-swufe.springeropen.com/articles/10.1186/s40854-024-00643-1).

In [4]:
# =============================================================================
# VARIABLE OBJETIVO
# =============================================================================

# Creamos la variable objetivo 'Target' según el movimiento direccional del precio
df_binance_1d['Target'] = (df_binance_1d['Close'].shift(-1) > df_binance_1d['Close']).astype(int)

# Revisamos las primeras filas
df_binance_1d[['Close', 'Target']].head(10)

,Close,Target
0,4285.08,0
1,4108.37,1
2,4139.98,0
3,4086.29,0
4,4016.00,1
5,4040.00,1
6,4114.01,1
7,4316.01,0
8,4280.68,1
9,4337.44,0


## 🔹 Indicadores Técnicos

En esta sección añadiremos **indicadores técnicos** al dataset diario de BTC/USD.  
Estos indicadores aportan información adicional sobre la **tendencia, momentum, volatilidad y volumen del mercado**, lo que puede mejorar la capacidad predictiva de nuestros modelos.

### 📝 Ventanas Seleccionadas

Para la **predicción diaria** de BTC/USD, las ventanas más importantes suelen ser las **medias y largas**, ya que:

- Capturan la tendencia general del mercado.
- Proporcionan contexto sobre la dirección del precio.
- Evitan que el modelo se sobreajuste a fluctuaciones de muy corto plazo.

Las ventanas **cortas** pueden aportar algo de sensibilidad a cambios recientes, pero su utilidad es limitada y, si se usan en exceso, pueden introducir **ruido** en la predicción.

Debido a ello, predominarán las **ventanas medias y largas** en los indicadores técnicos seleccionados, que se presentan a continuación. Esto asegura que los indicadores capturen la tendencia diaria relevante sin introducir demasiado ruido de corto plazo.

### 📈 Indicadores de Tendencia

Estos indicadores ayudan a identificar la dirección general del mercado y posibles cambios de tendencia. Para la predicción diaria hemos seleccionado los siguientes:

- **SMA (Simple Moving Average)**: medias móviles simples de **20 y 50 días**, que representan tendencias de medio y largo plazo.
- **EMA (Exponential Moving Average)**: medias móviles exponenciales de **20 y 50 días**, más sensibles a los cambios recientes de precio.
- **MACD (Moving Average Convergence Divergence)**: con parámetros clásicos **12, 26, 9**, útil para identificar cambios de impulso en la tendencia.
- **Parabolic SAR**: indicador de stop and reverse, que señala posibles puntos de cambio de dirección de la tendencia.
- **KAMA (Kaufman Adaptive Moving Average)**: media móvil adaptativa con **n=10 días**, que ajusta su suavizado según la volatilidad reciente, capturando cambios rápidos de tendencia a corto plazo.




In [5]:
# =============================================================================
# INDICADORES DE TENDENCIA - Daily Optimizado
# =============================================================================

# --- SMA: medio plazo ---
for w in [20, 50]:
    df_binance_1d[f'SMA_{w}'] = df_binance_1d['Close'].rolling(window=w).mean()
    df_binance_1d[f'Close_SMA_ratio_{w}'] = df_binance_1d['Close'] / df_binance_1d[f'SMA_{w}']

# --- EMA: medio plazo ---
for w in [20, 50]:
    df_binance_1d[f'EMA_{w}'] = df_binance_1d['Close'].ewm(span=w, adjust=False).mean()
    df_binance_1d[f'Close_EMA_ratio_{w}'] = df_binance_1d['Close'] / df_binance_1d[f'EMA_{w}']

# --- MACD clásico (12,26,9) ---
macd = ta.trend.MACD(close=df_binance_1d['Close'], window_slow=26, window_fast=12, window_sign=9)
df_binance_1d['MACD'] = macd.macd()
df_binance_1d['MACD_signal'] = macd.macd_signal()
df_binance_1d['MACD_diff'] = macd.macd_diff()

# --- Parabolic SAR ---
df_binance_1d['PSAR'] = ta.trend.PSARIndicator(
    high=df_binance_1d['High'],
    low=df_binance_1d['Low'],
    close=df_binance_1d['Close'],
    step=0.02,
    max_step=0.2
).psar()

# --- KAMA: calculado manualmente ---
def KAMA(close, n=10, fast=2, slow=30):
    close = close.values
    kama = np.zeros_like(close)
    kama[:n] = close[:n].mean()  # inicializamos con la media de los primeros n valores
    
    fastest = 2 / (fast + 1)
    slowest = 2 / (slow + 1)
    
    for i in range(n, len(close)):
        change = abs(close[i] - close[i - n])
        volatility = np.sum(np.abs(close[i - n + 1:i + 1] - close[i - n:i]))
        ER = 0 if volatility == 0 else change / volatility
        SC = (ER * (fastest - slowest) + slowest) ** 2
        kama[i] = kama[i-1] + SC * (close[i] - kama[i-1])
    
    return pd.Series(kama, index=df_binance_1d.index)

df_binance_1d['KAMA_10'] = KAMA(df_binance_1d['Close'], n=10)
df_binance_1d['Close_KAMA_ratio'] = df_binance_1d['Close'] / df_binance_1d['KAMA_10']

df_binance_1d

,Open time,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,...,EMA_20,Close_EMA_ratio_20,EMA_50,Close_EMA_ratio_50,MACD,MACD_signal,MACD_diff,PSAR,KAMA_10,Close_KAMA_ratio
0,2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,3.454770e+06,3427.0,616.248541,2.678216e+06,...,4285.080000,1.000000,4285.080000,1.000000,NaN,NaN,NaN,4285.080000,4172.386000,1.027009
1,2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,5.086958e+06,5233.0,972.868710,4.129123e+06,...,4268.250476,0.962542,4278.150196,0.960315,NaN,NaN,NaN,4108.370000,4172.386000,0.984657
2,2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,1.549484e+06,2153.0,274.336042,1.118002e+06,...,4256.034240,0.972732,4272.731757,0.968930,NaN,NaN,NaN,4485.390000,4172.386000,0.992233
3,2017-08-20,4139.98,4211.08,4032.62,4086.29,467.083022,1.930364e+06,2321.0,376.795947,1.557401e+06,...,4239.868122,0.963778,4265.420316,0.958004,NaN,NaN,NaN,4472.682200,4172.386000,0.979365
4,2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,2.797232e+06,3972.0,557.356107,2.255663e+06,...,4218.547349,0.951986,4255.639127,0.943689,NaN,NaN,NaN,4460.228556,4172.386000,0.962519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913,2025-08-08,117472.02,117630.00,115878.71,116674.74,10045.211700,1.172188e+09,2095268.0,4584.864500,5.349658e+08,...,115879.186313,1.006865,113413.353729,1.028757,334.011549,652.395213,-318.383664,112034.020000,115481.389765,1.010334
2914,2025-08-09,116674.74,117944.05,116299.13,116462.25,9514.131440,1.111838e+09,1390951.0,4869.854160,5.692283e+08,...,115934.716188,1.004550,113532.918289,1.025802,353.478721,592.611915,-239.133193,112257.859200,115497.449218,1.008353
2915,2025-08-10,116462.25,119311.11,116460.63,119294.01,14322.770730,1.694199e+09,2317065.0,8023.435130,9.490825e+08,...,116254.648932,1.026144,113758.843454,1.048657,590.597917,592.209115,-1.611198,112599.030648,115678.119659,1.031258
2916,2025-08-11,119294.27,122335.16,118050.11,118686.00,26494.334290,3.194612e+09,4021743.0,13243.429460,1.596640e+09,...,116486.206177,1.018885,113952.065279,1.041543,721.142381,617.995768,103.146613,113135.996996,116016.479514,1.023010


### 2. Momentum

In [6]:
# =============================================================================
# INDICADORES DE MOMENTUM (4 más importantes)
# =============================================================================

## 1. Williams %R (14 días)
#df_bitcoin['WILLR_14'] = df_bitcoin.ta.willr(length=14)
#
## 2. RSI (14 días)
#df_bitcoin['RSI_14'] = df_bitcoin.ta.rsi(length=14)
#
## 3. Stochastic Oscillator (14, 3) - Genera múltiples columnas
#stoch = df_bitcoin.ta.stoch(k=14, d=3)
#df_bitcoin = pd.concat([df_bitcoin, stoch], axis=1)
## Las columnas generadas son: STOCHk_14_3_3 y STOCHd_14_3_3
#
## 4. Rate of Change (12 y 14 días)
#df_bitcoin['ROC_12'] = df_bitcoin.ta.roc(length=12)
#df_bitcoin['ROC_14'] = df_bitcoin.ta.roc(length=14)

### 3. Volatilidad

In [7]:
# =============================================================================
# INDICADORES DE VOLATILIDAD (4 más importantes)
# =============================================================================

# 1. Bollinger Bands (20,2)
#bbands = ta.bbands(df_bitcoin['Close'], length=20, std=2)
#df_bitcoin = pd.concat([df_bitcoin, bbands], axis=1)
#
## Revisar los nombres reales de las columnas generadas
#bb_lower = [col for col in bbands.columns if 'BBL' in col][0]
#bb_middle = [col for col in bbands.columns if 'BBM' in col][0]
#bb_upper = [col for col in bbands.columns if 'BBU' in col][0]
#
## Posición del precio entre las bandas (0-100%)
#df_bitcoin['BB_Position'] = ((df_bitcoin['Close'] - df_bitcoin[bb_lower]) /
#                             (df_bitcoin[bb_upper] - df_bitcoin[bb_lower]) * 100)
#
## Ancho de las bandas normalizado
#df_bitcoin['BB_Width'] = ((df_bitcoin[bb_upper] - df_bitcoin[bb_lower]) /
#                          df_bitcoin[bb_middle] * 100)
#
## 2. Average True Range (14 días)
#df_bitcoin['ATR_14'] = ta.atr(high=df_bitcoin['High'], low=df_bitcoin['Low'], close=df_bitcoin['Close'], length=14)
#
## 3. Keltner Channels (20 períodos)
#kc = ta.kc(high=df_bitcoin['High'], low=df_bitcoin['Low'], close=df_bitcoin['Close'], length=20)
#df_bitcoin = pd.concat([df_bitcoin, kc], axis=1)
## Columnas generadas: KCLe_20_2 (Lower), KCBe_20_2 (Basis/Middle), KCUe_20_2 (Upper)
#
## 4. Donchian Channels (20 días)
#donchian = ta.donchian(high=df_bitcoin['High'], low=df_bitcoin['Low'], close=df_bitcoin['Close'], lower_length=20, upper_length=20)
#df_bitcoin = pd.concat([df_bitcoin, donchian], axis=1)
## Columnas generadas: DCL_20_20 (Lower), DCM_20_20 (Middle), DCU_20_20 (Upper)

### 4. Volumen

In [8]:
# =============================================================================
# INDICADORES DE VOLUMEN (4 más importantes)
# =============================================================================

# 1. On Balance Volume (OBV)
#df_bitcoin['OBV'] = df_bitcoin.ta.obv()
#
## 2. Chaikin Money Flow (21 días)
#df_bitcoin['CMF_21'] = df_bitcoin.ta.cmf(length=21)
#
## 3. Accumulation/Distribution Line
#df_bitcoin['AD'] = df_bitcoin.ta.ad()
#
## 4. Money Flow Index (14 días)
#df_bitcoin['MFI_14'] = df_bitcoin.ta.mfi(length=14)

### 5. Características adicionales

In [9]:
# =============================================================================
# CARACTERÍSTICAS ADICIONALES RECOMENDADAS
# =============================================================================

# Variables de precio - retornos logarítmicos
#df_bitcoin['Returns_1d'] = df_bitcoin.ta.log_return(length=1)
#df_bitcoin['Returns_2d'] = df_bitcoin.ta.log_return(length=2)
#df_bitcoin['Returns_5d'] = df_bitcoin.ta.log_return(length=5)
#
## Rango normalizado (High-Low)/Close
#df_bitcoin['Range_Norm'] = (df_bitcoin['High'] - df_bitcoin['Low']) / df_bitcoin['Close']
#
## Gap: (Open - Previous_Close) / Previous_Close
#df_bitcoin['Gap'] = (df_bitcoin['Open'] - df_bitcoin['Close'].shift(1)) / df_bitcoin['Close'].shift(1)
#
## Variables temporales
#df_bitcoin['Day_of_Week'] = df_bitcoin.index.dayofweek
#df_bitcoin['Month'] = df_bitcoin.index.month
#
## Ratios útiles para capturar posición relativa
#df_bitcoin['Price_to_SMA7'] = df_bitcoin['Close'] / df_bitcoin['SMA_7']
#df_bitcoin['Price_to_SMA20'] = df_bitcoin['Close'] / df_bitcoin['SMA_20']
#df_bitcoin['Price_to_EMA12'] = df_bitcoin['Close'] / df_bitcoin['EMA_12']
#
## Ratio entre SMAs (señal de tendencia)
#df_bitcoin['SMA7_to_SMA20'] = df_bitcoin['SMA_7'] / df_bitcoin['SMA_20']
#
## ATR normalizado por precio
#df_bitcoin['ATR_Normalized'] = df_bitcoin['ATR_14'] / df_bitcoin['Close']
#
## Pendientes para capturar cambios de tendencia (últimos 3 días)
#df_bitcoin['OBV_Slope_3d'] = df_bitcoin['OBV'].diff(3)
#df_bitcoin['RSI_Slope_3d'] = df_bitcoin['RSI_14'].diff(3)
#df_bitcoin['MACD_Slope_3d'] = df_bitcoin['MACD_12_26_9'].diff(3)
#
## Volumen normalizado
#df_bitcoin['Volume_Ratio'] = df_bitcoin['Volume'] / df_bitcoin['Volume'].rolling(20).mean()
#
## Distancias a niveles clave
#df_bitcoin['Distance_to_PSAR'] = (df_bitcoin['Close'] - df_bitcoin['PSAR']) / df_bitcoin['Close']